In [1]:
from collections import Counter

import pandas as pd
import statbotics

sb = statbotics.Statbotics()

In [6]:
all_matches = []
events_1 = sb.get_events(year=2022, week=1, fields=["key"])
events_2 = sb.get_events(year=2022, week=2, fields=["key"])
events_3 = sb.get_events(year=2022, week=3, fields=["key"])
events_4 = sb.get_events(year=2022, week=4, fields=["key"])
events_5 = sb.get_events(year=2022, week=5, fields=["key"])
events_6 = sb.get_events(year=2022, week=6, fields=["key"])
for week, events in enumerate([[], events_1, events_2, events_3, events_4, events_5, events_6]):
    for event in events:
        try:
            matches = sb.get_matches(event=event["key"])
            for match in matches:
                match["week"] = week
            all_matches.extend(matches)
        except Exception:
            pass
matches_df = pd.DataFrame(all_matches)
matches_df.head()

,key,year,event,comp_level,set_number,match_number,playoff,time,status,red,...,blue_1,red_2,blue_2,red_endgame,blue_endgame,red_fouls,blue_fouls,red_no_fouls,blue_no_fouls,week
0,2022bcvi_qm1,2022,2022bcvi,qm,1,1,False,1646426638,Completed,"7332,4421,7787",...,3.0,0.0,0.0,15.0,6.0,0.0,4.0,35.0,11.0,1
1,2022bcvi_qm2,2022,2022bcvi,qm,1,2,False,1646427312,Completed,"7287,7190,8338",...,18.0,0.0,0.0,0.0,4.0,0.0,0.0,7.0,26.0,1
2,2022bcvi_qm3,2022,2022bcvi,qm,1,3,False,1646428102,Completed,"216,7167,7173",...,13.0,0.0,0.0,0.0,6.0,0.0,0.0,2.0,25.0,1
3,2022bcvi_qm4,2022,2022bcvi,qm,1,4,False,1646428873,Completed,"6485,244,8082",...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,28.0,6.0,1
4,2022bcvi_qm5,2022,2022bcvi,qm,1,5,False,1646429430,Completed,"288,7167,216",...,14.0,0.0,0.0,6.0,0.0,0.0,0.0,23.0,20.0,1


In [17]:
out = []
for _, m in matches_df.iterrows():
    for alliance in ["red", "blue"]:
        try:
            out.append({
                "teams": [int(x) for x in m[alliance].split(",")],
                "event": m.event,
                "week": m.week,
                "comp_level": m.comp_level,
                "set_number": m.set_number,
                "match_number": m.match_number,
                "alliance": alliance,
                "auto": int(m[alliance + "_auto"]),
                "teleop": int(m[alliance + "_teleop"]),
                "endgame": int(m[alliance + "_endgame"]),
                "score": int(m[alliance + "_no_fouls"]),
                "foul_score": int(m[alliance + "_score"]),
                "cargo_rp": bool(m[alliance + "_rp_1"]),
                "hanger_rp": bool(m[alliance + "_rp_2"]),
            })
        except Exception:
            pass
    
all_matches_df = pd.DataFrame(out)

In [18]:
matches_100_df = all_matches_df[all_matches_df.score >= 100]

print(matches_df.shape[0])
print(matches_100_df.shape[0])
print()

print(matches_df.groupby("week").agg("count")["red"].tolist())
print(matches_100_df.groupby("week").agg("count")["teams"].tolist())
print()

for score in range(100, 210, 10):
    print(str(score) + "-" + str(score + 9), end="\t\t")
    for week in range(1, 7):
        temp_df = matches_100_df[(matches_100_df.score >= score) & (matches_100_df.score < score + 10) & (matches_100_df.week == week)]
        print(temp_df.shape[0], end="\t")
    print()

12946
1045

[1775, 1995, 2135, 2622, 2000, 2419]
[38, 33, 92, 236, 230, 416]

100-109		22	23	46	117	104	182	
110-119		10	7	24	58	65	121	
120-129		3	3	13	29	28	65	
130-139		2	0	3	17	21	27	
140-149		1	0	4	5	7	11	
150-159		0	0	1	5	5	3	
160-169		0	0	1	2	0	6	
170-179		0	0	0	2	0	1	
180-189		0	0	0	1	0	0	
190-199		0	0	0	0	0	0	
200-209		0	0	0	0	0	0	


In [27]:
# Top score by 1690, 1577 FMS Error
display(matches_100_df.sort_values(by=["score"], ascending=False).head(20))

teams = []
for t in matches_100_df.teams:
    teams.extend(t)
    
print(len(dict(Counter(teams))))
    
sorted([x for x in dict(Counter(teams)).items() if x[1] >= 20], key=lambda x:-x[1])

,teams,event,week,comp_level,set_number,match_number,alliance,auto,teleop,endgame,score,foul_score,cargo_rp,hanger_rp
13225,"[1690, 4338, 1577]",2022iscmp,4,qm,1,62,blue,36,117,30,183,191,True,True
21594,"[1678, 254, 7419]",2022casj,6,qf,1,1,red,26,108,45,179,179,False,False
11814,"[973, 2551, 254]",2022cada,4,qm,1,3,red,24,113,36,173,177,True,True
11830,"[1678, 254, 766]",2022cada,4,qm,1,11,red,34,108,30,172,176,True,True
11988,"[1678, 254, 5458]",2022cada,4,sf,1,2,red,28,106,34,168,180,False,False
25480,"[8019, 4206, 6800]",2022txcmp1,6,qm,1,79,red,30,108,30,168,172,True,True
21578,"[1678, 4973, 254]",2022casj,6,qm,1,83,red,30,108,30,168,168,True,True
25071,"[2910, 7034, 2811]",2022pncmp,6,qm,1,84,blue,34,100,30,164,164,True,True
21602,"[1678, 254, 7419]",2022casj,6,qf,1,2,red,34,84,45,163,167,False,False
8964,"[6740, 1577, 1690]",2022isde3,3,qm,1,40,red,36,102,25,163,163,True,True


946


[(1690, 37),
 (2910, 36),
 (1678, 33),
 (1577, 31),
 (1323, 30),
 (973, 27),
 (148, 26),
 (254, 26),
 (3310, 24),
 (2046, 24),
 (6800, 22),
 (27, 21),
 (5940, 21)]

In [22]:
# Both autos > 38 are FMS errors
display(all_matches_df[all_matches_df.auto >= 38].sort_values(by=["auto", "score"], ascending=False).head(10))
display(all_matches_df[all_matches_df.teleop >= 100].sort_values(by=["teleop"], ascending=False).head(10))
display(all_matches_df[all_matches_df.endgame >= 45].sort_values(by=["endgame", "score"], ascending=False).head(10))

,teams,event,week,comp_level,set_number,match_number,alliance,auto,teleop,endgame,score,foul_score,cargo_rp,hanger_rp
11328,"[1296, 7540, 7091]",2022txfor,3,qm,1,43,red,52,6,0,58,66,False,False
25696,"[3005, 3310, 3035]",2022txcmp2,6,sf,1,1,red,42,67,30,139,147,False,False
13214,"[3075, 1577, 2231]",2022iscmp,4,qm,1,57,red,38,93,30,161,169,True,True
18796,"[67, 27, 4384]",2022mitry,5,sf,1,1,red,38,68,36,142,142,False,False
8926,"[7039, 5614, 1690]",2022isde3,3,qm,1,21,red,38,72,31,141,153,True,True
13262,"[1690, 1577, 3065]",2022iscmp,4,qf,1,1,red,38,72,30,140,148,False,False
24248,"[2590, 5895, 1923]",2022mrcmp,6,sf,1,3,red,38,70,31,139,143,False,False
17284,"[1671, 5940, 3255]",2022cafr,5,qm,1,68,red,38,84,16,138,138,True,True
20252,"[2252, 4028, 4576]",2022tnkn,5,qm,1,70,red,38,64,36,138,138,True,True
13150,"[3065, 1577, 5654]",2022iscmp,4,qm,1,25,red,38,68,30,136,140,True,True


,teams,event,week,comp_level,set_number,match_number,alliance,auto,teleop,endgame,score,foul_score,cargo_rp,hanger_rp
13225,"[1690, 4338, 1577]",2022iscmp,4,qm,1,62,blue,36,117,30,183,191,True,True
11814,"[973, 2551, 254]",2022cada,4,qm,1,3,red,24,113,36,173,177,True,True
11830,"[1678, 254, 766]",2022cada,4,qm,1,11,red,34,108,30,172,176,True,True
17308,"[973, 1323, 5817]",2022cafr,5,qf,1,2,red,34,108,15,157,181,False,False
21578,"[1678, 4973, 254]",2022casj,6,qm,1,83,red,30,108,30,168,168,True,True
21594,"[1678, 254, 7419]",2022casj,6,qf,1,1,red,26,108,45,179,179,False,False
25480,"[8019, 4206, 6800]",2022txcmp1,6,qm,1,79,red,30,108,30,168,172,True,True
11988,"[1678, 254, 5458]",2022cada,4,sf,1,2,red,28,106,34,168,180,False,False
17408,"[7840, 3476, 7157]",2022cala,5,qm,1,41,red,28,106,25,159,159,True,True
8920,"[2212, 1690, 3075]",2022isde3,3,qm,1,18,red,32,103,21,156,160,True,True


,teams,event,week,comp_level,set_number,match_number,alliance,auto,teleop,endgame,score,foul_score,cargo_rp,hanger_rp
21594,"[1678, 254, 7419]",2022casj,6,qf,1,1,red,26,108,45,179,179,False,False
21602,"[1678, 254, 7419]",2022casj,6,qf,1,2,red,34,84,45,163,167,False,False
25596,"[2687, 3310, 5431]",2022txcmp2,6,qm,1,40,red,34,81,45,160,160,True,True
24392,"[1533, 587, 3459]",2022nccmp,6,qf,1,2,red,28,41,45,114,118,False,False
24408,"[1533, 587, 3459]",2022nccmp,6,f,1,1,red,30,38,45,113,117,False,False
8601,"[2358, 2481, 2338]",2022ilpe,3,qm,1,18,blue,22,40,45,107,107,True,True
17053,"[2202, 6421, 2506]",2022wimi,4,f,1,2,blue,30,30,45,105,105,False,False
11342,"[8842, 4153, 3005]",2022txfor,3,qm,1,50,red,18,39,45,102,102,True,True
16153,"[4476, 781, 4678]",2022on326,4,qm,1,9,blue,26,30,45,101,101,True,True
5680,"[2834, 503, 240]",2022mimil,2,qf,4,2,red,32,22,45,99,103,False,False


In [30]:
qm_matches_100_df = matches_100_df[matches_100_df.comp_level == "qm"]
display(qm_matches_100_df.sort_values(by=["score"], ascending=False).head(20))

teams = []
for t in qm_matches_100_df.teams:
    teams.extend(t)
sorted([x for x in dict(Counter(teams)).items() if x[1] >= 10], key=lambda x:-x[1])

,teams,event,week,comp_level,set_number,match_number,alliance,auto,teleop,endgame,score,foul_score,cargo_rp,hanger_rp
13225,"[1690, 4338, 1577]",2022iscmp,4,qm,1,62,blue,36,117,30,183,191,True,True
11814,"[973, 2551, 254]",2022cada,4,qm,1,3,red,24,113,36,173,177,True,True
11830,"[1678, 254, 766]",2022cada,4,qm,1,11,red,34,108,30,172,176,True,True
21578,"[1678, 4973, 254]",2022casj,6,qm,1,83,red,30,108,30,168,168,True,True
25480,"[8019, 4206, 6800]",2022txcmp1,6,qm,1,79,red,30,108,30,168,172,True,True
25071,"[2910, 7034, 2811]",2022pncmp,6,qm,1,84,blue,34,100,30,164,164,True,True
8964,"[6740, 1577, 1690]",2022isde3,3,qm,1,40,red,36,102,25,163,163,True,True
25855,"[6844, 2122, 2486]",2022utwv,6,qm,1,73,blue,26,100,36,162,162,True,True
13214,"[3075, 1577, 2231]",2022iscmp,4,qm,1,57,red,38,93,30,161,169,True,True
25596,"[2687, 3310, 5431]",2022txcmp2,6,qm,1,40,red,34,81,45,160,160,True,True


[(1690, 21),
 (1323, 18),
 (973, 17),
 (1678, 16),
 (2910, 16),
 (1577, 15),
 (254, 14),
 (3310, 11),
 (148, 11),
 (5940, 10),
 (2046, 10)]

In [31]:
elim_matches_100_df = matches_100_df[matches_100_df.comp_level != "qm"]
display(elim_matches_100_df.sort_values(by=["score"], ascending=False).head(20))

teams = []
for t in elim_matches_100_df.teams:
    teams.extend(t)
sorted([x for x in dict(Counter(teams)).items() if x[1] >= 12], key=lambda x:-x[1])

,teams,event,week,comp_level,set_number,match_number,alliance,auto,teleop,endgame,score,foul_score,cargo_rp,hanger_rp
21594,"[1678, 254, 7419]",2022casj,6,qf,1,1,red,26,108,45,179,179,False,False
11988,"[1678, 254, 5458]",2022cada,4,sf,1,2,red,28,106,34,168,180,False,False
21602,"[1678, 254, 7419]",2022casj,6,qf,1,2,red,34,84,45,163,167,False,False
11968,"[1678, 254, 5458]",2022cada,4,qf,1,1,red,36,92,30,158,162,False,False
21614,"[1678, 254, 7419]",2022casj,6,sf,1,1,red,34,84,40,158,158,False,False
16536,"[148, 3310, 4610]",2022txirv,4,sf,1,1,red,32,96,30,158,158,False,False
17308,"[973, 1323, 5817]",2022cafr,5,qf,1,2,red,34,108,15,157,181,False,False
11976,"[1678, 254, 5458]",2022cada,4,qf,1,2,red,32,90,34,156,164,False,False
11994,"[1678, 254, 5458]",2022cada,4,f,1,2,red,36,90,30,156,160,False,False
25128,"[4911, 2910, 3218]",2022pncmp,6,sf,1,2,red,34,86,30,150,154,False,False


[(2910, 20),
 (1678, 17),
 (1577, 16),
 (1690, 16),
 (148, 15),
 (2046, 14),
 (4414, 13),
 (27, 13),
 (6800, 13),
 (3310, 13),
 (254, 12),
 (1323, 12)]

In [39]:
elim_matches_100_df.apply({"teams": lambda x: frozenset(x), "score": lambda x: x, "event": lambda x: x, "week": lambda x: x}).groupby(["event", "teams"]).agg({"week": "count", "score": "mean"}).sort_values(by=["score", "week"], ascending=False).head(20)

,,week,score
event,teams,,
2022cada,"(5458, 1678, 254)",6,153.166667
2022casj,"(7419, 1678, 254)",6,153.000000
2022cafr,"(5817, 1323, 973)",6,140.166667
2022isde1,"(6104, 1577, 1690)",2,137.000000
2022txirv,"(4610, 148, 3310)",4,134.250000
2022isde3,"(1576, 1577, 1690)",6,129.333333
2022cada,"(973, 8020, 7157)",2,128.500000
2022pncmp,"(3218, 2910, 4911)",7,127.571429
2022mitry,"(27, 4384, 67)",7,127.285714


In [50]:
out = []
for _, m in matches_df.iterrows():
    try:
        out.append({
            "red_teams": [int(x) for x in m["red"].split(",")],
            "blue_teams": [int(x) for x in m["blue"].split(",")],
            "event": m.event,
            "week": m.week,
            "comp_level": m.comp_level,
            "set_number": m.set_number,
            "match_number": m.match_number,
            "alliance": alliance,
            "red_score": int(m["red_no_fouls"]),
            "red_cargo_rp": bool(m["red_rp_1"]),
            "red_hanger_rp": bool(m["red_rp_2"]),
            "blue_score": int(m["blue_no_fouls"]),
            "blue_cargo_rp": bool(m["blue_rp_1"]),
            "blue_hanger_rp": bool(m["blue_rp_2"]),
            "score_diff": int(m["red_no_fouls"]) - int(m["blue_no_fouls"]),
            "score_sum": int(m["red_no_fouls"]) + int(m["blue_no_fouls"]),
            "score_min": min(int(m["red_no_fouls"]), int(m["blue_no_fouls"]))
        })
    except Exception:
        pass
        
all_matches_both_df = pd.DataFrame(out)

In [51]:
matches_both_100_df = all_matches_both_df[(all_matches_both_df.red_score >= 100) & (all_matches_both_df.blue_score >= 100)]

In [52]:
display(all_matches_both_df.sort_values(by=["score_sum"], ascending=False).head(20))

,red_teams,blue_teams,event,week,comp_level,set_number,match_number,alliance,red_score,red_cargo_rp,red_hanger_rp,blue_score,blue_cargo_rp,blue_hanger_rp,score_diff,score_sum,score_min
12660,"[148, 6800, 1745]","[3847, 624, 7616]",2022txcmp,6,qf,1,1,blue,144,False,False,117,False,False,27,261,117
12798,"[2687, 3310, 5431]","[7506, 8573, 3005]",2022txcmp2,6,qm,1,40,blue,160,True,True,101,True,True,59,261,101
10812,"[1678, 254, 7419]","[604, 971, 8404]",2022casj,6,f,1,2,blue,136,False,False,124,False,False,12,260,124
10811,"[1678, 254, 7419]","[604, 971, 8404]",2022casj,6,f,1,1,blue,143,False,False,116,False,False,27,259,116
5997,"[1678, 254, 5458]","[7157, 973, 5274]",2022cada,4,f,1,2,blue,156,False,False,103,False,False,53,259,103
12127,"[2590, 5895, 1923]","[341, 2539, 1914]",2022mrcmp,6,f,1,2,blue,131,False,False,126,False,False,5,257,126
12535,"[955, 2521, 4488]","[2910, 7034, 2811]",2022pncmp,6,qm,1,84,blue,93,True,True,164,True,True,-71,257,93
12853,"[3005, 3310, 3035]","[3847, 624, 7616]",2022txcmp2,6,f,1,2,blue,128,False,False,127,False,False,1,255,127
12943,"[2122, 359, 6844]","[4499, 2486, 3166]",2022utwv,6,f,1,2,blue,128,False,False,126,False,False,2,254,126
6607,"[3075, 1577, 2231]","[3388, 7845, 7039]",2022iscmp,4,qm,1,57,blue,161,True,True,91,True,True,70,252,91


In [53]:
display(all_matches_both_df.sort_values(by=["score_min"], ascending=False).head(20))

,red_teams,blue_teams,event,week,comp_level,set_number,match_number,alliance,red_score,red_cargo_rp,red_hanger_rp,blue_score,blue_cargo_rp,blue_hanger_rp,score_diff,score_sum,score_min
12853,"[3005, 3310, 3035]","[3847, 624, 7616]",2022txcmp2,6,f,1,2,blue,128,False,False,127,False,False,1,255,127
12943,"[2122, 359, 6844]","[4499, 2486, 3166]",2022utwv,6,f,1,2,blue,128,False,False,126,False,False,2,254,126
12127,"[2590, 5895, 1923]","[341, 2539, 1914]",2022mrcmp,6,f,1,2,blue,131,False,False,126,False,False,5,257,126
10812,"[1678, 254, 7419]","[604, 971, 8404]",2022casj,6,f,1,2,blue,136,False,False,124,False,False,12,260,124
12511,"[2990, 1425, 2521]","[360, 4911, 6350]",2022pncmp,6,qm,1,60,blue,121,True,True,130,True,True,-9,251,121
12660,"[148, 6800, 1745]","[3847, 624, 7616]",2022txcmp,6,qf,1,1,blue,144,False,False,117,False,False,27,261,117
10146,"[2252, 2614, 5002]","[2393, 4028, 3492]",2022tnkn,5,sf,1,2,blue,116,False,False,116,False,False,0,232,116
10811,"[1678, 254, 7419]","[604, 971, 8404]",2022casj,6,f,1,1,blue,143,False,False,116,False,False,27,259,116
11063,"[1746, 8736, 3344]","[6829, 5632, 2974]",2022gacmp,6,qm,1,17,blue,116,True,False,123,True,True,-7,239,116
9047,"[118, 179, 7094]","[2992, 364, 5872]",2022lake,5,f,1,2,blue,120,False,False,115,False,False,5,235,115
